In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName("Linear").getOrCreate();

23/12/26 23:28:38 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.25.126.173 instead (on interface eth0)
23/12/26 23:28:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/26 23:28:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/26 23:28:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.csv('./data/ML/Ecommerce_Customers.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)


In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol='features')

In [11]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)


In [15]:
final_data = output.select('features', 'Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean|  500.9201265761463|
| stddev|  80.68349545690265|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+


In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean| 495.45724114228415|
| stddev|   76.0574029158347|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+


In [24]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [25]:
lr_model = lr.fit(train_data)

23/12/26 23:38:36 WARN Instrumentation: [217db1f5] regParam is zero, which might cause numerical instability and overfitting.
23/12/26 23:38:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [27]:
test_result = lr_model.evaluate(test_data)

In [29]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.415942865411921|
|-0.5526235381649371|
|  10.10827743108473|
| -4.815434075903056|
| -4.738878261550724|
|  4.223732186517168|
|-22.228398010982232|
|-0.9866100798600428|
|-3.8382884073846526|
| 20.925271806725277|
|  3.366650218168047|
| -6.531786452289225|
| -5.827533996815532|
|-15.416772892025392|
|  6.142449211103838|
| -8.278994069339774|
| -6.698402087576767|
| -12.05575084193788|
|  7.562286371717789|
| -9.520735789057824|
+-------------------+


In [31]:
test_result.rootMeanSquaredError

10.858879796962867

In [32]:
test_result.r2

0.9794764942732458

In [33]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+


In [35]:
unlabeled_data = test_data.select('features')

In [36]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.0472221394875...|
|[31.1239743499119...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3584771924370...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5741380228732...|
|[31.6548096756927...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8209982016720...|
|[31.8279790554652...|
+--------------------+


In [37]:
predictions = lr_model.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 331.3448126686055|
|[30.5743636841713...| 442.6170372962306|
|[30.7377203726281...|451.67246476514515|
|[30.8364326747734...|472.31733450289266|
|[30.8794843441274...| 494.9454782464054|
|[31.0472221394875...| 388.2736670025042|
|[31.1239743499119...|  509.175451850748|
|[31.2606468698795...| 422.3132413368114|
|[31.2681042107507...| 427.3088215812086|
|[31.2834474760581...| 570.8558176189422|
|[31.3584771924370...|491.80930023130736|
|[31.5147378578019...|496.34427444875064|
|[31.5257524169682...| 449.7931608066974|
|[31.5741380228732...| 559.8260450526122|
|[31.6548096756927...| 469.1209745164447|
|[31.7207699002873...| 547.0539275473627|
|[31.7242025238451...|510.08628937553726|
|[31.8093003166791...|  548.827650204779|
|[31.8209982016720...|417.11299464149556|
|[31.8279790554652...|449.52348333599934|
+--------------------+------------

In [39]:
test_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[30.3931845423455...|  319.9288698031936|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8364326747734...|  467.5019004269896|
|[30.8794843441274...|  490.2065999848547|
|[31.0472221394875...|  392.4973991890214|
|[31.1239743499119...|  486.9470538397658|
|[31.2606468698795...|  421.3266312569514|
|[31.2681042107507...|  423.4705331738239|
|[31.2834474760581...|  591.7810894256675|
|[31.3584771924370...|  495.1759504494754|
|[31.5147378578019...|  489.8124879964614|
|[31.5257524169682...|  443.9656268098819|
|[31.5741380228732...|  544.4092721605869|
|[31.6548096756927...|  475.2634237275485|
|[31.7207699002873...|   538.774933478023|
|[31.7242025238451...|  503.3878872879605|
|[31.8093003166791...|  536.7718993628412|
|[31.8209982016720...| 424.67528101321335|
|[31.8279790554652...|  440.0027475469415|
+----------

In [40]:
predictions.describe().show()

+-------+-----------------+
|summary|       prediction|
+-------+-----------------+
|  count|              147|
|   mean|495.1792684109492|
| stddev|76.92184982954022|
|    min|252.7558998908944|
|    max|753.2526726750373|
+-------+-----------------+


In [41]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean| 495.45724114228415|
| stddev|   76.0574029158347|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+
